In [1]:
import kagglehub

path = kagglehub.dataset_download("akshat103/e-waste-image-dataset")

print("Path to dataset files:", path)

C:\Users\DELL\anaconda3\envs\ewastesem6\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\DELL\.cache\kagglehub\datasets\akshat103\e-waste-image-dataset\versions\1


In [5]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torchvision import datasets, transforms 
from torch.utils.data import DataLoader 
import matplotlib.pyplot as plt 
import numpy as np 
import os

In [6]:
dataset_dir=r"C:\Users\DELL\OneDrive\Desktop\modified-dataset"

In [7]:
print("Dataset Directory:", dataset_dir)
print("Subdirectories:", os.listdir(dataset_dir))

Dataset Directory: C:\Users\DELL\OneDrive\Desktop\modified-dataset
Subdirectories: ['test', 'train', 'val']


In [8]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=12):
        super(CNNModel, self).__init__()

        self.conv1=nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.pool=nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1=nn.Dropout(p=0.2)
        
        self.conv2=nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.dropout2=nn.Dropout(p=0.3)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.dropout3=nn.Dropout(p=0.3)
        
        self.flatten=nn.Flatten()
        self.fc1=nn.Linear(64*28*28, 128)
        self.dropout4=nn.Dropout(p=0.4)
        self.fc2=nn.Linear(128, num_classes) 

    def forward(self,x):
        x=self.pool(torch.relu(self.conv1(x)))
        x=self.dropout1(x)
        x=self.pool(torch.relu(self.conv2(x)))
        x=self.dropout2(x)
        x=self.pool(torch.relu(self.conv3(x)))
        x=self.dropout3(x)
        x=self.flatten(x)
        x=torch.relu(self.fc1(x))
        x=self.dropout4(x)
        x=self.fc2(x)
        return x

device="cpu"
model=CNNModel().to(device)
print(model)

CNNModel(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.2, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout2): Dropout(p=0.3, inplace=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout3): Dropout(p=0.3, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (dropout4): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=128, out_features=12, bias=True)
)


In [12]:
transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset_path=r"C:\Users\DELL\OneDrive\Desktop\modified-dataset"
train_dataset=datasets.ImageFolder(root=dataset_path+'/train', transform=transform)
val_dataset=datasets.ImageFolder(root=dataset_path+'/val', transform=transform)
test_dataset=datasets.ImageFolder(root=dataset_path+'/test', transform=transform)

batch_size=32 
train_loader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader=DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Classes: ", train_dataset.classes)

Classes:  ['Battery', 'Keyboard', 'Microwave', 'Mobile', 'Mouse', 'PCB', 'Player', 'Printer', 'Television', 'Washing Machine']


In [16]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001, )

num_epochs=50

In [17]:
for epoch in range (num_epochs):
    model.train()
    running_loss=0.0
    correct_train=0
    total_train=0

    for images, labels in train_loader:
        images, labels=images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs=model(images)
        loss=criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
        running_loss+=loss.item()

        _,predicted=torch.max(outputs, 1)
        correct_train+=(predicted==labels).sum().item()
        total_train+=labels.size(0)
    train_loss=running_loss/len(train_loader)
    train_accuracy=100*correct_train/total_train

    model.eval()
    val_loss=0.0
    correct_val=0
    total_val=0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels=images.to(device), labels.to(device)
            outputs=model(images)
            loss=criterion(outputs, labels)
            val_loss+=loss.item()

            _, predicted=torch.max(outputs, 1)
            correct_val+=(predicted==labels).sum().item()
            total_val+=labels.size(0)

    val_loss/=len(val_loader)
    val_accuracy=100*correct_val/total_val

    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% - "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

print("Training done")


Epoch [1/50] - Train Loss: 1.8272, Train Acc: 36.67% - Val Loss: 1.7212, Val Acc: 44.33%
Epoch [2/50] - Train Loss: 1.5699, Train Acc: 46.42% - Val Loss: 1.4085, Val Acc: 53.33%
Epoch [3/50] - Train Loss: 1.3464, Train Acc: 53.25% - Val Loss: 1.3944, Val Acc: 62.00%
Epoch [4/50] - Train Loss: 1.1705, Train Acc: 60.21% - Val Loss: 1.2894, Val Acc: 60.33%
Epoch [5/50] - Train Loss: 0.9653, Train Acc: 66.83% - Val Loss: 1.2994, Val Acc: 59.33%
Epoch [6/50] - Train Loss: 0.8111, Train Acc: 71.75% - Val Loss: 1.1984, Val Acc: 63.67%
Epoch [7/50] - Train Loss: 0.6490, Train Acc: 77.71% - Val Loss: 1.2155, Val Acc: 62.67%
Epoch [8/50] - Train Loss: 0.5293, Train Acc: 82.92% - Val Loss: 1.3031, Val Acc: 60.67%
Epoch [9/50] - Train Loss: 0.4483, Train Acc: 85.00% - Val Loss: 1.2471, Val Acc: 63.33%
Epoch [10/50] - Train Loss: 0.3526, Train Acc: 87.75% - Val Loss: 1.3060, Val Acc: 63.33%
Epoch [11/50] - Train Loss: 0.3437, Train Acc: 87.92% - Val Loss: 1.3060, Val Acc: 63.67%
Epoch [12/50] - Tra